In [1]:
from __future__ import absolute_import, division, print_function
import logging, os, sys

# Enable logging
logging.basicConfig(format='[%(levelname)s] %(message)s', level=logging.INFO, stream=sys.stdout)

# Re-import packages if they change
%load_ext autoreload
%autoreload 2

# Recursion Depth
sys.setrecursionlimit(1000000000)

# Intialize tqdm to always use the notebook progress bar
import tqdm

tqdm.tqdm = tqdm.tqdm_notebook

# Third-party libraries
import comet_ml

import numpy as np
import pandas as pd
import nilearn.plotting as nip
import matplotlib.pyplot as plt
import nibabel as nib
import numpy as np
import torch
import collections
%matplotlib inline
plt.rcParams["figure.figsize"] = (12,6)
%config InlineBackend.figure_format='retina'  # adapt plots for retina displays
import git
import aneurysm_utils.evaluation as evaluation

# Project utils

import aneurysm_utils
from aneurysm_utils import evaluation, training,preprocessing


/home/tu-ramonabendias/.pyenv/versions/3.8.10/lib/python3.8/site-packages/nilearn/datasets/__init__.py:86: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
if "workspace" in os.getcwd():
    ROOT = "/workspace" # local 
elif "/group/cake" in os.getcwd(): 
    ROOT = "/group/cake" # Jupyter Lab


In [3]:
env = aneurysm_utils.Environment(project="our-git-project", root_folder=ROOT)
env.cached_data["comet_key"] = "EGrR4luSis87yhHbs2rEaqAWs" 
env.print_info()

Environment Info:

Library Version: 0.1.0
Configured Project: our-git-project

Folder Structure: 
- Root folder: /group/cake
 - Project folder: /group/cake/our-git-project
 - Datasets folder: /data/training
 - Models folder: /group/cake/our-git-project/models
 - Experiments folder: /group/cake/our-git-project/experiments


In [4]:
dataset_params = {
    "prediction": "mask",
    "mri_data_selection": "", 
    "balance_data": False,
    "seed": 1,
    "resample_voxel_dim": (1.2, 1.2, 1.2)
}

preprocessing_params = {
    'min_max_normalize': True,
    'mean_std_normalize': False,
    'smooth_img': False, # can contain a number: smoothing factor
    'intensity_segmentation': False
}


In [5]:
from aneurysm_utils.data_collection import load_aneurysm_dataset

df = load_aneurysm_dataset(
    env,
    mri_data_selection=dataset_params["mri_data_selection"],
    random_state=dataset_params["seed"]
)
df.head()

,Aneurysm Geometry,Angiography Data,Vessel Geometry,Labeled Mask Index,Location,Age,Sex,Rupture Status,Age Bin,Aneurysm Count,Case,Path Orig,Path Mask,Path Vessel,Path Labeled Mask
0,A001.stl,A001_orig.nii.gz,A001_vessel.stl,1,Acom,48,m,1.0,"(40, 50]",1,A001,/data/training/A001_orig.nii.gz,/data/training/A001_masks.nii.gz,/data/training/A001_vessel.nii.gz,/data/training/A001_labeledMasks.nii.gz
1,A003.stl,A003_orig.nii.gz,A003_vessel.stl,1,Pcom,58,f,0.0,"(50, 60]",1,A003,/data/training/A003_orig.nii.gz,/data/training/A003_masks.nii.gz,/data/training/A003_vessel.nii.gz,/data/training/A003_labeledMasks.nii.gz
2,A005.stl,A005_orig.nii.gz,A005_vessel.stl,1,PICA,45,m,1.0,"(40, 50]",1,A005,/data/training/A005_orig.nii.gz,/data/training/A005_masks.nii.gz,/data/training/A005_vessel.nii.gz,/data/training/A005_labeledMasks.nii.gz
3,A006.stl,A006_orig.nii.gz,A006_vessel.stl,1,ACom,46,f,1.0,"(40, 50]",1,A006,/data/training/A006_orig.nii.gz,/data/training/A006_masks.nii.gz,/data/training/A006_vessel.nii.gz,/data/training/A006_labeledMasks.nii.gz
4,A008.stl,A008_orig.nii.gz,A008_vessel.stl,1,ACA,72,f,0.0,"(70, 80]",1,A008,/data/training/A008_orig.nii.gz,/data/training/A008_masks.nii.gz,/data/training/A008_vessel.nii.gz,/data/training/A008_labeledMasks.nii.gz


In [6]:
# Load MRI images and split into train, test, and validation
from aneurysm_utils.data_collection import split_mri_images
case_list = ["A130_R","A118","A120","A115","A133","A073","A072","A077","A064"]
df = df.loc[df["Case"].isin(case_list)]

train_data, test_data,val_data, _ = split_mri_images(
    env, 
    df, 
    prediction=dataset_params["prediction"], 
    encode_labels=False,
    random_state=dataset_params["seed"],
    balance_data=dataset_params["balance_data"],
    resample_voxel_dim=dataset_params["resample_voxel_dim"]
)

mri_imgs_train, labels_train, train_participants = train_data
mri_imgs_test, labels_test, test_participants = test_data
mri_imgs_val, labels_val, val_participants= val_data

9
8
         Images
-----  --------
All           9
Train         7
Val           1
Test          1



  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [7]:
mri_imgs_test = mri_imgs_test + mri_imgs_train + mri_imgs_val
labels_test = labels_test + labels_train + labels_val

In [8]:
from aneurysm_utils import preprocessing

most_commen_shape = preprocessing.check_mri_shapes(mri_imgs_test)

Most common:
(116, 116, 100):       9


In [9]:
size = most_commen_shape  #(139, 139, 120)

test_index = [i for i, e in enumerate(mri_imgs_test) if e.shape != size]
mri_imgs_test = [i for j, i in enumerate(mri_imgs_test) if j not in test_index]
labels_test = [i for j, i in enumerate(labels_test) if j not in test_index]


In [10]:
def patch_creater(image, patch_size, test = None):
    """
    Creates overlapping patches from  preprocessed image, the number of patches is fixed to certain value
    and the size can be changed as well
    ----------
    image: numpy.array
        image which will be sliced into patches
    patch_size: tuple of int
        size of the patch, equal in each direction
   
    Returns
    -------
    numpy.array  (n_patches,channels,patch_size,patch_size,patch_size)
        list containing the patches

    """
    
    


    dim = np.array(image.shape)# size of the image
    n_patches = np.ceil(dim/patch_size).astype(int) # calculates the number of patches for each dim, to cover all voxel at least once
    rest  = n_patches * patch_size%dim ## calculates the remaining voxel which are going to overlap 

    patches = []
    for i in range(n_patches[0]):
        
        if i == n_patches[0]-1: ## only the last cube is an overlapped cube
            start_x = i*patch_size-rest[0]## indices were to start and stop the slicing true the image array
            stop_x= (i+1)* patch_size-rest[0]
              
        else:    
            start_x = i*patch_size
            stop_x = (i+1)* patch_size

        
              
        for j in range(n_patches[1]):
            if j == n_patches[1]-1: ## only the last cube is an overlapped cube
                start_y = j*patch_size-rest[1]
                stop_y= (j+1)* patch_size-rest[1]
              
            else:    
                start_y = j*patch_size
                stop_y = (j+1)* patch_size
            
            for k in range(n_patches[2]):
                if k == n_patches[2]-1: 
                    start_z = k*patch_size-rest[2]
                    stop_z = (k+1)* patch_size-rest[2]
              
                else:    
                    start_z = k*patch_size
                    stop_z = (k+1)* patch_size

                if test:
                    
                    patches.append(torch.from_numpy(image[start_x:stop_x,start_y:stop_y,start_z:stop_z]))
                else:
                    patches.append(image[start_x:stop_x,start_y:stop_y,start_z:stop_z])
        
    #return np.array([*patches])
    return patches

In [11]:
def patch_list(data,patch_size,test = None):
    """
    data: numpy.array
        containing dataset of dimensions (size_of_set,height,width,depth),e.g. (75,139,139,120)
    patch_size: int
    
    Return
    
    list_patch: list
        each element is one image of type numpy.array/torch.tensor with dimensions(n_classes,most_common_shape),
    """
    list_patch = []

    
    for n in range(len(data)):
        patch = patch_creater(data[n],patch_size,test)
        list_patch = list_patch+patch
    

    return list_patch

In [12]:
from aneurysm_utils import preprocessing
patch_size = 64
size_of_test = len(mri_imgs_test)

# preprocess all lists as one to have a working mean_std_normalization
mri_imgs_test = preprocessing.preprocess(env, mri_imgs_test, preprocessing_params)
###creating patches
mri_imgs_test = np.asarray(mri_imgs_test)
mri_imgs_test = patch_list(mri_imgs_test,patch_size)

# preprocess mask
x, y, h = labels_train[0].shape
labels_test = patch_list(labels_test,patch_size)

[INFO] Preprocessing: Min Max Normalize...


In [13]:
from aneurysm_utils.utils.pytorch_utils import predict
#from aneurysm_utils.models.unet_3d_oktay import unet_3D
from aneurysm_utils.models.attention_unet import unet_grid_attention_3D

In [14]:
model_name = "2021-07-13-15-03-01_mask-pytorch-attention-unet"
weights_name = "Attention_Unet_Attention_Unet_4920.pt"

In [15]:
model = unet_grid_attention_3D()
    
#model = unet_grid_attention_3D(feature_scale=4, n_classes=2, is_deconv=True, in_channels=1,nonlocal_mode='concatenation', attention_dsample=(2,2,2), is_batchnorm=True)
PATH = f"../../experiments/{model_name}/{weights_name}"

device = torch.device('cuda')


model.load_state_dict(torch.load(PATH, map_location=device))

<All keys matched successfully>

In [16]:
len(mri_imgs_test)

72

In [17]:
from aneurysm_utils.utils import pytorch_utils
from torch.utils.data.dataloader import DataLoader
test_dataset = pytorch_utils.PytorchDataset(
                mri_imgs_test,
                labels = labels_test,
                dtype=np.float64,
                segmentation=True
                
            )



In [18]:
test_loader = DataLoader(
            test_dataset,
            batch_size=1,  # TODO: use fixed batch size of 5
            shuffle=False,
            num_workers=0,
            pin_memory=True,
        )
predictions = predict(model, test_loader, apply_softmax=False )

nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
Default upsampling behavior when mode=trilinear is changed to align_corners=False since 0.4.0. Please specify align_corners=True if the old behavior is desired. See the documentation of nn.Upsample for details.


In [19]:
###############   Name und Ort gegebenfalls ändern
np.save(f"preds_attention_ramona_{model_name}_{weights_name}.npy",predictions)

In [20]:
np.load(f"preds_attention_ramona_{model_name}_{weights_name}.npy")

array([[[[[0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ]],

         [[0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          [0.        , 0.        , 0.        , ..., 0.        ,
           0.        , 0.        ],
          ...,
          [0.        , 0.        , 0.        , ..., 0.        ,
    